### Модули эффективного кода: functools, itertools, operator, dataclasses 

Модуль functools в Python — это стандартный модуль, содержащий утилиты для работы с функциями и функциями высшего порядка (т.е. функциями, которые принимают или возвращают другие функции).

Он очень часто используется для оптимизации, мемоизации, частичного применения аргументов и декорирования.


1. functools.lru_cache
Позволяет кэшировать результаты функции, чтобы при повторных вызовах с теми же аргументами не пересчитывать заново.

lru_cache — декоратор, который кэширует результаты функции по аргументам. Если кэш заполнен, выбрасывает (evict) наименее недавно использованный (LRU) элемент.

cache — то же самое, что lru_cache(maxsize=None): неограниченный кэш (без вытеснения).

In [38]:
from functools import lru_cache, cache

# @cache
@lru_cache(maxsize=2**8)
def fib(n):
    if n < 2:
        return n
    return fib(n-1) + fib(n-2)


print(fib(100))
print(fib.cache_info())
print(fib.cache_parameters())
print(fib.__wrapped__(100))
print(fib.cache_clear())

@lru_cache(typed=True)
def f(x):
    return x * 2

f(1)
f(1.0)
print(f.cache_info())
print(f.cache_parameters())

354224848179261915075
CacheInfo(hits=98, misses=101, maxsize=256, currsize=101)
{'maxsize': 256, 'typed': False}
354224848179261915075
None
CacheInfo(hits=0, misses=2, maxsize=128, currsize=2)
{'maxsize': 128, 'typed': True}


2. functools.partial

Создаёт новую функцию с заранее подставленными аргументами.

In [ ]:
from functools import partial

def power(base, degree):
    return base**degree

square = partial(power, degree=2, base=6)
cube = partial(power, 3)
chetvorka = partial(power, base=5)

print(square())
print(cube(2))
print(chetvorka(degree=2))

36
9
25


functools.reduce()

Назначение:
Постепенно «сворачивает» (reduce = «свернуть») последовательность в одно значение, применяя заданную функцию к элементам по очереди.

Синтаксис:
```python
from functools import reduce
reduce(function, iterable[, initializer])
```

function — функция, которая принимает два аргумента (например, lambda x, y: x + y).

iterable — последовательность (список, кортеж и т.д.).

initializer (необязательно) — начальное значение аккумулятора.

In [ ]:
from functools import reduce

numbers = [1, 2, 3, 4]
result = reduce(lambda a, b: a * b, numbers, 4)
print(result)  # 96 (4*1*2*3*4)

96



functools.wraps — это удобный декоратор в Python, который используется при написании собственных декораторов. Его основная задача — сохранять метаданные исходной функции (например, имя, документацию, аннотации), когда вы оборачиваете её другим декоратором. Без wraps обернутая функция теряет свои оригинальные свойства и вместо этого принимает имя и документацию обёртки.

In [42]:
from functools import wraps

def my_decorator(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        print("Вызов функции")
        return func(*args, **kwargs)
    return wrapper

@my_decorator
def greet(name):
    """Приветствие пользователя"""
    return f"Hello, {name}!"

print(greet.__name__)  # greet
print(greet.__doc__)   # Приветствие пользователя


def my_decorator2(func):
    def wrapper(*args, **kwargs):
        print("Вызов функции")
        return func(*args, **kwargs)
    return wrapper

@my_decorator2
def greet2(name):
    """Приветствие пользователя"""
    return f"Hello, {name}!"

print(greet2.__name__)  # wrapper
print(greet2.__doc__)   # None


greet
Приветствие пользователя
wrapper
None


Проблема, которую решает cmp_to_key

В Python 2 функции сортировки (sorted(), list.sort()) принимали параметр cmp — то есть функцию сравнения двух элементов:

In [46]:
from functools import cmp_to_key

def compare(a, b):
    if a > b:
        return 1
    if a < b:
        return -1
    return 0

numbers = [5, 2, 9, 1, 10, 11, -1, -2, 1, 2]
sorted_numbers = sorted(numbers, key=cmp_to_key(compare))
print(sorted_numbers)  # [1, 2, 5, 9]


[-2, -1, 1, 1, 2, 2, 5, 9, 10, 11]


singledispatch реализует generic function на базе типа первого аргумента. Ты объявляешь одну «общую» функцию, а потом регистрируешь для неё реализации для конкретных типов. При вызове singledispatch выбирает самую подходящую реализацию по типу первого аргумента (с учётом наследования).

In [50]:
from functools import singledispatch

@singledispatch
def show(obj):
    print("Объект:", obj)

@show.register(int)
def _(obj):
    print("Это число:", obj)

@show.register(list)
def _(obj):
    print("Это список:", obj)

show(10)        # Это число: 10
show([1, 2, 3]) # Это список: [1, 2, 3]
show("текст")   # Объект: текст


class Animal: pass
class Dog(Animal): pass
class Cat(Animal): pass
class Bulldog(Dog): pass


@singledispatch
def sound(x):
    print("generic animal")

@sound.register(Animal)
def _(x):
    print("some animal sound")

@sound.register(Dog)
def _(x):
    print("woof")

@sound.register(Cat)
def _(x):
    print("meow")


sound(Animal())   # some animal sound
sound(Dog())      # woof
sound(Cat())      # meow
sound(Bulldog())  # woof потому что Dog - родительский класс Bulldog (ближайший)
#никаких множественных наследований

Это число: 10
Это список: [1, 2, 3]
Объект: текст
some animal sound
woof
meow
woof


Модуль itertools в Python — это стандартная библиотека, которая содержит функции для работы с итераторами, позволяя создавать эффективные, компактные и ленивые (lazy) последовательности данных.

Ключевые особенности:

- Генерирует элементы по мере необходимости, что экономит память.

- Содержит как бесконечные итераторы, так и функции для комбинирования, фильтрации и группировки элементов.

- Позволяет писать короткий и выразительный код для перебора и обработки последовательностей.

**Infinite iterators:**

In [63]:
from itertools import cycle, count, repeat


for i in count(start=10, step=2):
    print(i)
    if i > 20:
        break

colors = ['red', 'green', 'blue']
col = cycle(colors)
for _ in range(6):
    print(next(col))


for i in repeat('hello', 5):
    print(i)
    

10
12
14
16
18
20
22
red
green
blue
red
green
blue
hello
hello
hello
hello
hello


Итераторы, работающие с входными данными

In [2]:
import itertools


for i in itertools.accumulate([1, 2, 3, 4, 5], max):
    print(i)

for i in itertools.accumulate([1, 2, 3, 4, 5]):
    print(i)
# создаёт последовательность накопленных значений.
# По умолчанию суммирует элементы. Можно использовать любую бинарную функцию (operator.mul, max и т.д.).



1
2
3
4
5
1
3
6
10
15


In [ ]:
a = list(itertools.chain("ABCFD", "DSFG"))
print(a) # объединяет несколько итерируемых объектов в один поток.

data = ['ABC', 'DEF'] # как chain, но принимает один итерируемый объект,
# содержащий другие итерируемые объекты.
print(list(itertools.chain.from_iterable(data)))

['A', 'B', 'C', 'F', 'D', 'D', 'S', 'F', 'G']
['A', 'B', 'C', 'D', 'E', 'F']


In [74]:
# фильтрует элементы из data, оставляя только те,
# где соответствующий элемент selectors равен True (или 1).

data = 'ABCDEF'
selectors = [1, 0, 1, 0, 1, 1]
print(list(itertools.compress(data, selectors)))

['A', 'C', 'E', 'F']


In [ ]:
# пропускает элементы пока predicate возвращает True, потом выдаёт все оставшиеся элементы.
data = [1, 4, 6, 3, 8]
result = list(itertools.dropwhile(lambda x: x < 5, data))
print(result)

data = [1, 4, 6, 3, 8] # оставляет только элементы, где predicate ложь
result = list(itertools.filterfalse(lambda x: x < 5, data))
print(result)

[6, 3, 8]
[6, 8]


In [2]:
import itertools

data = [1, 1, 2, 2, 2, 3, 1] # Группирует соседние элементы итератора,
# у которых одинаковое значение (по какому-то критерию).
for key, group in itertools.groupby(data):
    print(key, list(group)) # groupby() группирует только подряд идущие одинаковые элементы!

print()

data = [1, 1, 2, 2, 2, 3, 1]
for key, group in itertools.groupby(sorted(data)):
    print(key, list(group))


1 [1, 1]
2 [2, 2, 2]
3 [3]
1 [1]

1 [1, 1, 1]
2 [2, 2, 2]
3 [3]


In [9]:
words = ["apple", "ant", "banana", "bear", "cherry"]
for key, group in itertools.groupby(sorted(words), key=lambda w: w[0]):
    print(key, list(group))

print()

words = ['abd', 'ed', 'w', 'efef', 'eff', 'qww', 'q']
for key, group in itertools.groupby(words, key=len):
    print(key, list(group))

a ['ant', 'apple']
b ['banana', 'bear']
c ['cherry']

3 ['abd']
2 ['ed']
1 ['w']
4 ['efef']
3 ['eff', 'qww']
1 ['q']


In [22]:
data = 'ABCDEFG'
print(list(itertools.islice(data, 2, 6, 2))) # срез элементов, 
# как seq[start:stop:step], но для любого итератора.

for x in itertools.islice(itertools.count(10, 2), 1, 5):
    print(x)

['C', 'E']
12
14
16
18


In [ ]:
nums = [1, 2, 3, 4, 5] # Создаёт пары соседних элементов из последовательности.
print(list(itertools.pairwise(nums)))

[(1, 2), (2, 3), (3, 4), (4, 5)]


In [ ]:
def sum_3(a, b, c):
    return a + b + c

posl = [(3, 4, 5), (1, 2, 3)]
# itertools.starmap - это функция, которая применяет другую функцию к элементам итерируемого объекта, 
# "распаковывая" каждый элемент как аргументы.
a = list(itertools.starmap(sum_3, posl))
print(a)

[12, 6]


In [3]:
# takewhile - это функция, которая возвращает элементы из итерируемого объекта, пока условие истинно.
#  Как только условие становится ложным, итерация останавливается.

numbers2 = [5, 3, 1, -2, 4, 6]
result2 = itertools.takewhile(lambda x: x > 0, numbers2)
print(list(result2))

[5, 3, 1]


In [ ]:
numbers = [10, 20, 30, 40]

# tee создает несколько независимых итераторов из одного исходного итерируемого объекта.
iter1, iter2, iter3 = itertools.tee(numbers, 3)

print(list(iter1))
print(list(iter2))
print(list(iter3))

[10, 20, 30, 40]
[10, 20, 30, 40]
[10, 20, 30, 40]


In [5]:
# zip_longest объединяет элементы из нескольких итерируемых объектов, 
# заполняя недостающие значения указанным заполнителем.

names = ["Alice", "Bob"]
ages = [25, 30, 35]
cities = ["NYC", "LA", "Chicago", "Miami"]

result = itertools.zip_longest(names, ages, cities, fillvalue="Unknown")
print(list(result))

[('Alice', 25, 'NYC'), ('Bob', 30, 'LA'), ('Unknown', 35, 'Chicago'), ('Unknown', 'Unknown', 'Miami')]


Комбинаторные итераторы

In [7]:
# Декартово произведение двух или более множеств
list1 = ['A', 'B']
list2 = [1, 2]
list3 = [3, 4]
result = itertools.product(list1, list2, list3)
print(list(result))

[('A', 1, 3), ('A', 1, 4), ('A', 2, 3), ('A', 2, 4), ('B', 1, 3), ('B', 1, 4), ('B', 2, 3), ('B', 2, 4)]


In [8]:
# Генерирует все возможные перестановки элементов заданной длины.
items = ['A', 'B', 'C']
result = itertools.permutations(items, 2)
print(list(result))

[('A', 'B'), ('A', 'C'), ('B', 'A'), ('B', 'C'), ('C', 'A'), ('C', 'B')]


In [ ]:
# Генерирует все возможные комбинации элементов заданной длины (без повторений).
items = ['A', 'B', 'C', 'D']
result = itertools.combinations(items, 2)
print(list(result))

[('A', 'B'), ('A', 'C'), ('A', 'D'), ('B', 'C'), ('B', 'D'), ('C', 'D')]


In [10]:
# Генерирует комбинации, где элементы могут повторяться.
items = ['A', 'B', 'C']
result = itertools.combinations_with_replacement(items, 2)
print(list(result))

[('A', 'A'), ('A', 'B'), ('A', 'C'), ('B', 'B'), ('B', 'C'), ('C', 'C')]


Библиотека operator позволяет использовать операции Python (+, -, *, >, < и т.д.) в виде функций, что особенно полезно в функциональном программировании.

In [11]:
import operator

In [12]:
# Базовые арифметические операции
print(operator.add(5, 3))       # 8 - сложение
print(operator.sub(10, 4))      # 6 - вычитание  
print(operator.mul(3, 4))       # 12 - умножение
print(operator.truediv(10, 3))  # 3.333... - деление
print(operator.floordiv(10, 3)) # 3 - целочисленное деление
print(operator.mod(10, 3))      # 1 - остаток от деления
print(operator.pow(2, 3))       # 8 - возведение в степень

# Унарные операции
print(operator.neg(5))          # -5 - смена знака
print(operator.pos(-3))         # -3 - унарный плюс
print(operator.abs(-10))        # 10 - модуль

8
6
12
3.3333333333333335
3
1
8
-5
-3
10


In [13]:
# Все операторы сравнения
print(operator.eq(5, 5))        # True - равно ==
print(operator.ne(5, 3))        # True - не равно !=
print(operator.lt(3, 5))        # True - меньше <
print(operator.le(3, 3))        # True - меньше или равно <=
print(operator.gt(5, 3))        # True - больше >
print(operator.ge(5, 5))        # True - больше или равно >=

# Проверка идентичности объектов
a = [1, 2]
b = [1, 2]
c = a
print(operator.is_(a, b))       # False - тот же объект?
print(operator.is_(a, c))       # True
print(operator.is_not(a, b))    # True - не тот же объект?

True
True
True
True
True
True
False
True
True


In [ ]:
# Логические операторы
print(operator.not_(True))      # False - логическое НЕ
print(operator.and_(True, False)) # False - логическое И
print(operator.or_(True, False))  # True - логическое ИЛИ
print(operator.xor(True, True))   # False - исключающее ИЛИ

# Операторы с образцом
data = {"name": "John", "age": 30}
print(operator.contains(data, "name"))  # True - проверка вхождения

False
False
True
False
True


In [ ]:
my_list = [1, 2, 3, 4, 5]

# Получение элемента по индексу
print(operator.getitem(my_list, 2))     # 3 - эквивалент my_list[2]

# Установка элемента по индексу
operator.setitem(my_list, 2, 99)
print(my_list)                         # [1, 2, 99, 4, 5]

# Удаление элемента по индексу
operator.delitem(my_list, 2)
print(my_list)                     # [1, 2, 4, 5]

# Конкатенация и повторение
print(operator.concat([1, 2], [3, 4]))  # [1, 2, 3, 4]  a + b

3
[1, 2, 99, 4, 5]
[1, 2, 4, 5]
[1, 2, 3, 4]
True


Dataclasses (классы данных) — это декоратор и модуль в Python, которые автоматически генерируют специальные методы для классов, что значительно сокращает шаблонный код.

In [19]:
class Person:
    def __init__(self, name, age, email):
        self.name = name
        self.age = age
        self.email = email
    
    def __repr__(self):
        return f"Person(name='{self.name}', age={self.age}, email='{self.email}')"
    
    def __eq__(self, other):
        if not isinstance(other, Person):
            return False
        return (self.name, self.age, self.email) == (other.name, other.age, other.email)

print(Person('name', 22, 'c@gmail'))

Person(name='name', age=22, email='c@gmail')


In [23]:
import dataclasses

@dataclasses.dataclass
class Person:
    name: str
    age: int
    email: str = 'None'

print(Person('name', 22))

Person(name='name', age=22, email='None')


Параметры декоратора @dataclass
init=True - генерировать __init__ (по умолчанию True)

repr=True - генерировать __repr__ (по умолчанию True)

eq=True - генерировать __eq__ (по умолчанию True)

order=False - генерировать методы сравнения (<, >, <=, >=)

frozen=False - сделать экземпляры неизменяемыми

slots=False - использовать __slots__ для оптимизации памяти

In [ ]:
@dataclasses.dataclass(slots=True, order=True, frozen=True)
class Person:
    name: str
    age: int
    email: str = 'None'

pers = Person('Bob', 22, 'smf@mail')
pers.name = "NonBob"


FrozenInstanceError: cannot assign to field 'name'

In [36]:
from typing import List


@dataclasses.dataclass
class Inventory:
    items: List[str] = dataclasses.field(default_factory=list)
    max_capacity: int = dataclasses.field(default=100, compare=False)  # не участвует в сравнении
    created_at: str = dataclasses.field(init=False, repr=False)  # не включать в __init__
    

inv = Inventory(["apple", "banana"])

print(inv)

print(dataclasses.astuple(pers))
print(dataclasses.asdict(pers))

Inventory(items=['apple', 'banana'], max_capacity=100)
('Bob', 22, 'smf@mail')
{'name': 'Bob', 'age': 22, 'email': 'smf@mail'}
